In [31]:
# Starting with the import of all dependencies 
import atlite
import math
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from atlite.gis import shape_availability, ExclusionContainer
import numpy as np
import os
from matplotlib.gridspec import GridSpec
import seaborn as sns
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


import cartopy.crs as ccrs
from cartopy.crs import PlateCarree as plate
import cartopy.io.shapereader as shpreader
from pyproj import Geod

import logging
import warnings

warnings.simplefilter('ignore')
logging.captureWarnings(False)
logging.basicConfig(level=logging.INFO)

from pathlib import Path

In [32]:
""" >> not in use 

# Define WGS84 as CRS:
geod = Geod('+a=6378137 +f=0.0033528106647475126')
# Plotting the administrative borders of the DRB riparians
plt.rcParams['figure.figsize'] = [7, 7]
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
countries = ['Serbia', 'Bosnia and Herz.', 'Montenegro'] # Defining the countries by name
shapes = world[world.name.isin(countries)].set_index('name') # Pulling their shapes out of the natural earth dataset 

shapes.plot(); #Plotting the figure
"""

" >> not in use \n\n# Define WGS84 as CRS:\ngeod = Geod('+a=6378137 +f=0.0033528106647475126')\n# Plotting the administrative borders of the DRB riparians\nplt.rcParams['figure.figsize'] = [7, 7]\nworld = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))\ncountries = ['Serbia', 'Bosnia and Herz.', 'Montenegro'] # Defining the countries by name\nshapes = world[world.name.isin(countries)].set_index('name') # Pulling their shapes out of the natural earth dataset \n\nshapes.plot(); #Plotting the figure\n"

In [33]:
# bounds = shapes.cascaded_union.buffer(1).bounds
# cutout = atlite.Cutout('DRB_2020', module='era5', bounds=bounds, time=slice('2020-01-01', '2020-12-31')) #Defining the year and time for the cutout

In [34]:
# # Defining the countries by their shape and the excluded land areas 
# serbia = shapes.loc[['Serbia']].geometry.to_crs(excluder.crs)
# bosnia = shapes.loc[['Bosnia and Herz.']].geometry.to_crs(excluder.crs)
# montenegro = shapes.loc[['Montenegro']].geometry.to_crs(excluder.crs)

Change the mask depending on country that you are calculating the values for (i.e. bosnia, montenegro, serbia)

In [35]:
# masked, transform = shape_availability(bosnia, excluder)
# eligible_share = masked.sum() * excluder.res**2 / bosnia.geometry.item().area #Excluding the land and calculating the share that is actually eligable 

In [36]:
# #bosnia.geometry.item().area
# bosnia.geometry.item().area
# #montenegro.geometry.item().area

In [37]:
# # Plotting the eligable share and specifying the share in % 
# fig, ax = plt.subplots()
# ax = show(masked, transform=transform, cmap='Oranges', ax=ax)
# bosnia.plot(ax=ax, edgecolor='k', color='none',figsize=(9, 7.94))
# ax.set_title(f'Eligible area (green) {eligible_share * 100:2.2f}%');
# plt.savefig('BA_2020_AG.png', dpi=300) #No need to plot it each time, can be plotted ones depending on the layers selected.

In [38]:
# # Putting the plott into the context of the cutout area with the long and lat grid on top of it
# fig, ax = plt.subplots()
# ax = show(masked, transform=transform, cmap='Oranges', ax=ax)
# serbia.plot(ax=ax, edgecolor='k', color='None')
# cutout.grid.to_crs(excluder.crs).plot(edgecolor='grey', color='None', ax=ax, ls=':')
# ax.set_title(f'Eligible area (green) {eligible_share * 100:2.2f}%');

In [39]:
# # Computing the availability matrix A. This is done for all 3 countries at ones, since shapes in clude all the DRB countries, and the excluder includes
# # land area that should be excluded.
# A = cutout.availabilitymatrix(shapes, excluder)
# A

# Load Country Boundaries and Eligible Land Share data

In [41]:
# Save local file for later usage
countries_gdf_path=Path('data/downloaded/countries.geojson')
countries_gdf=gpd.read_file(countries_gdf_path)

print(f"""
    Country boundaries with eligible land share loaded.
    Source: '{countries_gdf_path}'
    {90 * '_'}
    Quick preview:\n 
    {countries_gdf}
    """)


    Country boundaries with eligible land share loaded.
    Source: 'data/downloaded/countries.geojson'
    __________________________________________________________________________________________
    Quick preview:
 
       ISO                 Country  eligible_land  \
0  SRB                  Serbia       0.261510   
1  BIH  Bosnia and Herzegovina       0.400597   
2  MNE              Montenegro       1.534151   

                                            geometry  
0  MULTIPOLYGON (((22.1207 42.3035, 22.0788 42.29...  
1  MULTIPOLYGON (((18.3141 42.6229, 18.2981 42.61...  
2  MULTIPOLYGON (((18.8482 42.2701, 18.8571 42.26...  
    


In [42]:
# # Here the average available land within each of the grid cells is computed. 
# fig, ax = plt.subplots(figsize=(10,10))
# A.sel(name='Serbia').plot(cmap='Oranges')
# shapes.loc[['Serbia']].plot(ax=ax, edgecolor='k', color='None')
# cutout.grid.plot(ax=ax, color='None', edgecolor='grey', ls=':')

Import the TIFF file from the Global Wind Atlas here by specifying the path to the file. IEC1, IEC2 and IEC3 should be read in for each of the DRB countries. 

In [ ]:
# Using rasterio, we import the BIH_capacity-factor_IEC3.tif which has been downloaded from the Global Wind Atlas (from here on refered to as GWA)
ds = rasterio.open("data/downloaded/GWA/SRB_capacity-factor_IEC3.tif", driver='GTiff')
wind_data = ds.read()
wind_data = np.nan_to_num(wind_data, nan= 0)
_,min_py,min_px = wind_data.shape
# Here we limit the number of pixels in x and y to be the number of pixels - 1, so that the calculation does not fall outside of it
min_py=min_py-1 
min_px=min_px-1

print(f"""
Capacity Factor (CF) - IEC3 data from Global Wind Atlas (GWA) loaded
Source: https://globalwindatlas.info/en/download/gis-files
Layer: CF_IEC3
""")


 Capacity Factor (CF) - IEC3 data from Global Wind Atlas (GWA) loaded
Source: https://globalwindatlas.info/en/download/gis-files
Layer: CF_IEC3



Chose the code below for Bosnia and Herzegovina

In [ ]:
bos = A.sel(name='Bosnia and Herz.') # We define the availability matrix for Bosnia and Herzegovina as bos
cf_arr = [] # We create an empty dictionary called cf_arr
N = 100 # We put 100 equally s
for j, y in enumerate(bos.y): #lat 
    cf_arr.append([]) #creating an empty array to store all the corresponding longitudes for that latitude
    for i, x in enumerate(bos.x): #long
        if bos[j,i] == 0:
            cf_arr[j].append(0.)
            continue
        else: 
            delta_x = abs(bos.x[i+1]-x)/2
            delta_y = abs(bos.y[j+1]-y)/2
            x_w = (x- delta_x)
            x_e = (x+ delta_x)
            y_n = (y + delta_y)
            y_s = (y - delta_y)
            lon_list, lat_list = np.linspace(x_w, x_e, N), np.linspace(y_s, y_n, N)
            #cf_arr[j].append({"avg": 0,"cf_arr":[]})
            cf_arr[j].append({"avg": 0,"cf_arr":[],'area_cell_avg':0})
            lats = np.array([y_n, y_s, y_s, y_n])
            lons = np.array([x_w, x_w, x_e, x_e]) 
            area, perim = geod.polygon_area_perimeter(lons, lats)
            cf_arr[j][i]["area_cell_avg"] = area / 1000000
            for lon in lon_list:
                for lat in lat_list:
                    py, px = ds.index(lon, lat)
                    cf_arr[j][i]["avg"] += wind_data[0,min(min_py,py),min(min_px,px)]/(N**2)
                    cf_arr[j][i]["cf_arr"].append(wind_data[0,min(min_py,py),min(min_px,px)])
                    #cf_arr[j][i]["area_cell_avg"]
                    #cf_arr[j][i]["area_cell_avg"] += (((111.32 * (math.cos((lat*math.pi)/180)))*0.25)**2)/(N**2) # store est_avg_area_of_cell_ij 
                    

# calculate land proportions per cell
# calculate c_avg per cell
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            tot = len(cf_arr[j][i]["cf_arr"]) # total points in the cell
            cf_arr[j][i]["area_cf_0.1"]= 0
            cf_arr[j][i]["area_cf_0.2"]= 0
            cf_arr[j][i]["area_cf_0.3"]= 0
            cf_arr[j][i]["area_cf_0.4"]= 0
            cf_arr[j][i]["area_cf_0.5"]= 0
            cf_arr[j][i]["avg_cf_0.1"]= 0
            cf_arr[j][i]["avg_cf_0.2"]= 0
            cf_arr[j][i]["avg_cf_0.3"]= 0
            cf_arr[j][i]["avg_cf_0.4"]= 0
            cf_arr[j][i]["avg_cf_0.5"]= 0
            for cf_ij in cf_arr[j][i]["cf_arr"]:
                if  cf_ij <= 0.1:
                    cf_arr[j][i]["area_cf_0.1"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.1"] += cf_ij
                    #continue
                elif cf_ij >0.1 and cf_ij <=0.2:
                    cf_arr[j][i]["area_cf_0.2"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.2"] += cf_ij
                elif cf_ij >0.2 and cf_ij <=0.3:
                    cf_arr[j][i]["area_cf_0.3"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.3"] += cf_ij
                elif cf_ij >0.3 and cf_ij <=0.4:
                    cf_arr[j][i]["area_cf_0.4"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.4"] += cf_ij
                elif cf_ij >0.4:
                    cf_arr[j][i]["area_cf_0.5"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.5"] += cf_ij

#finish cf_avg per cell
eps = 1e-10 #We turn the 0 into a number that is almost zero 
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            cf_arr[j][i]["avg_cf_0.1"] = cf_arr[j][i]["avg_cf_0.1"] / (tot * cf_arr[j][i]["area_cf_0.1"]+eps)
            cf_arr[j][i]["avg_cf_0.2"] = cf_arr[j][i]["avg_cf_0.2"] / (tot * cf_arr[j][i]["area_cf_0.2"]+eps)
            cf_arr[j][i]["avg_cf_0.3"] = cf_arr[j][i]["avg_cf_0.3"] / (tot * cf_arr[j][i]["area_cf_0.3"]+eps)
            cf_arr[j][i]["avg_cf_0.4"] = cf_arr[j][i]["avg_cf_0.4"] / (tot * cf_arr[j][i]["area_cf_0.4"]+eps)
            cf_arr[j][i]["avg_cf_0.5"] = cf_arr[j][i]["avg_cf_0.5"] / (tot * cf_arr[j][i]["area_cf_0.5"]+eps)

# Final average per wind technology type
cf_T_01 = 0
cf_T_02 = 0
cf_T_03 = 0
cf_T_04 = 0
cf_T_05 = 0

tot_01_weights = 0 # total area for cf_0.1 interval
tot_02_weights = 0 # total area for cf_0.2 interval
tot_03_weights = 0 # total area for cf_0.3 interval
tot_04_weights = 0 # total area for cf_0.4 interval
tot_05_weights = 0 # total area for cf_0.5 interval

# modify by land area availabilty per square (from atlite) 
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            tot_01_weights += cf_arr[j][i]["area_cf_0.1"]*bos[j,i]
            tot_02_weights += cf_arr[j][i]["area_cf_0.2"]*bos[j,i]
            tot_03_weights += cf_arr[j][i]["area_cf_0.3"]*bos[j,i]
            tot_04_weights += cf_arr[j][i]["area_cf_0.4"]*bos[j,i]
            tot_05_weights += cf_arr[j][i]["area_cf_0.5"]*bos[j,i]


# Calculate average capacity factor for each tech
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            cf_T_01 += cf_arr[j][i]["area_cf_0.1"] *bos[j,i] * cf_arr[j][i]["avg_cf_0.1"] / tot_01_weights
            cf_T_02 += cf_arr[j][i]["area_cf_0.2"] *bos[j,i] * cf_arr[j][i]["avg_cf_0.2"] / tot_02_weights
            cf_T_03 += cf_arr[j][i]["area_cf_0.3"] *bos[j,i] * cf_arr[j][i]["avg_cf_0.3"] / tot_03_weights
            cf_T_04 += cf_arr[j][i]["area_cf_0.4"] *bos[j,i] * cf_arr[j][i]["avg_cf_0.4"] / tot_04_weights
            cf_T_05 += cf_arr[j][i]["area_cf_0.5"] *bos[j,i] * cf_arr[j][i]["avg_cf_0.5"] / tot_05_weights

area_01_weights = 0 # total area for cf_0.1 interval
area_02_weights = 0 # total area for cf_0.2 interval
area_03_weights = 0 # total area for cf_0.3 interval
area_04_weights = 0 # total area for cf_0.4 interval
area_05_weights = 0 # total area for cf_0.5 interval


#A_cell = 408.5 #0.25*0.25 degrees = 111.1/4 = 27.7775 * 27.7775  
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            area_01_weights += cf_arr[j][i]["area_cf_0.1"]
            area_02_weights += cf_arr[j][i]["area_cf_0.2"]
            area_03_weights += cf_arr[j][i]["area_cf_0.3"]
            area_04_weights += cf_arr[j][i]["area_cf_0.4"]
            area_05_weights += cf_arr[j][i]["area_cf_0.5"]

# Calculate max capacity per tech (based on available land for that CF interval in each cell)
cap_T_01 = 0
cap_T_02 = 0
cap_T_03 = 0
cap_T_04 = 0
cap_T_05 = 0

for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            cap_T_01 += cf_arr[j][i]["area_cf_0.1"] * bos[j,i] * cf_arr[j][i]["area_cell_avg"] 
            cap_T_02 += cf_arr[j][i]["area_cf_0.2"] * bos[j,i] * cf_arr[j][i]["area_cell_avg"] 
            cap_T_03 += cf_arr[j][i]["area_cf_0.3"] * bos[j,i] * cf_arr[j][i]["area_cell_avg"] 
            cap_T_04 += cf_arr[j][i]["area_cf_0.4"] * bos[j,i] * cf_arr[j][i]["area_cell_avg"] 
            cap_T_05 += cf_arr[j][i]["area_cf_0.5"] * bos[j,i] * cf_arr[j][i]["area_cell_avg"] 



Chose the code below for Montenegro

In [ ]:
mon = A.sel(name='Montenegro') # We define the availability matrix for Montenegro as mon
cf_arr = [] # We create an empty dictionary called cf_arr
N = 100 # We put 100 equally s
for j, y in enumerate(mon.y): #lat 
    cf_arr.append([]) #creating an empty array to store all the corresponding longitudes for that latitude
    for i, x in enumerate(mon.x): #long
        if mon[j,i] == 0:
            cf_arr[j].append(0.)
            continue
        else: 
            delta_x = abs(mon.x[i+1]-x)/2
            delta_y = abs(mon.y[j+1]-y)/2
            x_w = (x- delta_x)
            x_e = (x+ delta_x)
            y_n = (y + delta_y)
            y_s = (y - delta_y)
            lon_list, lat_list = np.linspace(x_w, x_e, N), np.linspace(y_s, y_n, N)
            #cf_arr[j].append({"avg": 0,"cf_arr":[]})
            cf_arr[j].append({"avg": 0,"cf_arr":[],'area_cell_avg':0})
            lats = np.array([y_n, y_s, y_s, y_n])
            lons = np.array([x_w, x_w, x_e, x_e]) 
            area, perim = geod.polygon_area_perimeter(lons, lats)
            cf_arr[j][i]["area_cell_avg"] = area / 1000000
            for lon in lon_list:
                for lat in lat_list:
                    py, px = ds.index(lon, lat)
                    cf_arr[j][i]["avg"] += wind_data[0,min(min_py,py),min(min_px,px)]/(N**2)
                    cf_arr[j][i]["cf_arr"].append(wind_data[0,min(min_py,py),min(min_px,px)])
                    #cf_arr[j][i]["area_cell_avg"]
                    #cf_arr[j][i]["area_cell_avg"] += (((111.32 * (math.cos((lat*math.pi)/180)))*0.25)**2)/(N**2) # store est_avg_area_of_cell_ij 
                    

# calculate land proportions per cell
# calculate c_avg per cell
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            tot = len(cf_arr[j][i]["cf_arr"]) # total points in the cell
            cf_arr[j][i]["area_cf_0.1"]= 0
            cf_arr[j][i]["area_cf_0.2"]= 0
            cf_arr[j][i]["area_cf_0.3"]= 0
            cf_arr[j][i]["area_cf_0.4"]= 0
            cf_arr[j][i]["area_cf_0.5"]= 0
            cf_arr[j][i]["avg_cf_0.1"]= 0
            cf_arr[j][i]["avg_cf_0.2"]= 0
            cf_arr[j][i]["avg_cf_0.3"]= 0
            cf_arr[j][i]["avg_cf_0.4"]= 0
            cf_arr[j][i]["avg_cf_0.5"]= 0
            for cf_ij in cf_arr[j][i]["cf_arr"]:
                if  cf_ij <= 0.1:
                    cf_arr[j][i]["area_cf_0.1"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.1"] += cf_ij
                    #continue
                elif cf_ij >0.1 and cf_ij <=0.2:
                    cf_arr[j][i]["area_cf_0.2"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.2"] += cf_ij
                elif cf_ij >0.2 and cf_ij <=0.3:
                    cf_arr[j][i]["area_cf_0.3"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.3"] += cf_ij
                elif cf_ij >0.3 and cf_ij <=0.4:
                    cf_arr[j][i]["area_cf_0.4"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.4"] += cf_ij
                elif cf_ij >0.4:
                    cf_arr[j][i]["area_cf_0.5"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.5"] += cf_ij

#finish cf_avg per cell
eps = 1e-10 #We turn the 0 into a number that is almost zero 
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            cf_arr[j][i]["avg_cf_0.1"] = cf_arr[j][i]["avg_cf_0.1"] / (tot * cf_arr[j][i]["area_cf_0.1"]+eps)
            cf_arr[j][i]["avg_cf_0.2"] = cf_arr[j][i]["avg_cf_0.2"] / (tot * cf_arr[j][i]["area_cf_0.2"]+eps)
            cf_arr[j][i]["avg_cf_0.3"] = cf_arr[j][i]["avg_cf_0.3"] / (tot * cf_arr[j][i]["area_cf_0.3"]+eps)
            cf_arr[j][i]["avg_cf_0.4"] = cf_arr[j][i]["avg_cf_0.4"] / (tot * cf_arr[j][i]["area_cf_0.4"]+eps)
            cf_arr[j][i]["avg_cf_0.5"] = cf_arr[j][i]["avg_cf_0.5"] / (tot * cf_arr[j][i]["area_cf_0.5"]+eps)

# Final average per wind technology type
cf_T_01 = 0
cf_T_02 = 0
cf_T_03 = 0
cf_T_04 = 0
cf_T_05 = 0

tot_01_weights = 0 # total area for cf_0.1 interval
tot_02_weights = 0 # total area for cf_0.2 interval
tot_03_weights = 0 # total area for cf_0.3 interval
tot_04_weights = 0 # total area for cf_0.4 interval
tot_05_weights = 0 # total area for cf_0.5 interval

# modify by land area availabilty per square (from atlite) 
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            tot_01_weights += cf_arr[j][i]["area_cf_0.1"]*mon[j,i]
            tot_02_weights += cf_arr[j][i]["area_cf_0.2"]*mon[j,i]
            tot_03_weights += cf_arr[j][i]["area_cf_0.3"]*mon[j,i]
            tot_04_weights += cf_arr[j][i]["area_cf_0.4"]*mon[j,i]
            tot_05_weights += cf_arr[j][i]["area_cf_0.5"]*mon[j,i]


# Calculate average capacity factor for each tech
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            cf_T_01 += cf_arr[j][i]["area_cf_0.1"] *mon[j,i] * cf_arr[j][i]["avg_cf_0.1"] / tot_01_weights
            cf_T_02 += cf_arr[j][i]["area_cf_0.2"] *mon[j,i] * cf_arr[j][i]["avg_cf_0.2"] / tot_02_weights
            cf_T_03 += cf_arr[j][i]["area_cf_0.3"] *mon[j,i] * cf_arr[j][i]["avg_cf_0.3"] / tot_03_weights
            cf_T_04 += cf_arr[j][i]["area_cf_0.4"] *mon[j,i] * cf_arr[j][i]["avg_cf_0.4"] / tot_04_weights
            cf_T_05 += cf_arr[j][i]["area_cf_0.5"] *mon[j,i] * cf_arr[j][i]["avg_cf_0.5"] / tot_05_weights

area_01_weights = 0 # total area for cf_0.1 interval
area_02_weights = 0 # total area for cf_0.2 interval
area_03_weights = 0 # total area for cf_0.3 interval
area_04_weights = 0 # total area for cf_0.4 interval
area_05_weights = 0 # total area for cf_0.5 interval


#A_cell = 408.5 #0.25*0.25 degrees = 111.1/4 = 27.7775 * 27.7775  
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            area_01_weights += cf_arr[j][i]["area_cf_0.1"]
            area_02_weights += cf_arr[j][i]["area_cf_0.2"]
            area_03_weights += cf_arr[j][i]["area_cf_0.3"]
            area_04_weights += cf_arr[j][i]["area_cf_0.4"]
            area_05_weights += cf_arr[j][i]["area_cf_0.5"]

# Calculate max capacity per tech (based on available land for that CF interval in each cell)
cap_T_01 = 0
cap_T_02 = 0
cap_T_03 = 0
cap_T_04 = 0
cap_T_05 = 0

for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            cap_T_01 += cf_arr[j][i]["area_cf_0.1"] * mon[j,i] * cf_arr[j][i]["area_cell_avg"] 
            cap_T_02 += cf_arr[j][i]["area_cf_0.2"] * mon[j,i] * cf_arr[j][i]["area_cell_avg"] 
            cap_T_03 += cf_arr[j][i]["area_cf_0.3"] * mon[j,i] * cf_arr[j][i]["area_cell_avg"] 
            cap_T_04 += cf_arr[j][i]["area_cf_0.4"] * mon[j,i] * cf_arr[j][i]["area_cell_avg"] 
            cap_T_05 += cf_arr[j][i]["area_cf_0.5"] * mon[j,i] * cf_arr[j][i]["area_cell_avg"] 

Chose the code below for Serbia

In [ ]:
srb = A.sel(name='Serbia') # We define the availability matrix for Serbia as srb
cf_arr = [] # We create an empty dictionary called cf_arr
N = 100 # We put 100 equally s
for j, y in enumerate(srb.y): #lat 
    cf_arr.append([]) #creating an empty array to store all the corresponding longitudes for that latitude
    for i, x in enumerate(srb.x): #long
        if srb[j,i] == 0:
            cf_arr[j].append(0.)
            continue
        else: 
            delta_x = abs(srb.x[i+1]-x)/2
            delta_y = abs(srb.y[j+1]-y)/2
            x_w = (x- delta_x)
            x_e = (x+ delta_x)
            y_n = (y + delta_y)
            y_s = (y - delta_y)
            lon_list, lat_list = np.linspace(x_w, x_e, N), np.linspace(y_s, y_n, N)
            #cf_arr[j].append({"avg": 0,"cf_arr":[]})
            cf_arr[j].append({"avg": 0,"cf_arr":[],'area_cell_avg':0})
            lats = np.array([y_n, y_s, y_s, y_n])
            lons = np.array([x_w, x_w, x_e, x_e]) 
            area, perim = geod.polygon_area_perimeter(lons, lats)
            cf_arr[j][i]["area_cell_avg"] = area / 1000000
            for lon in lon_list:
                for lat in lat_list:
                    py, px = ds.index(lon, lat)
                    cf_arr[j][i]["avg"] += wind_data[0,min(min_py,py),min(min_px,px)]/(N**2)
                    cf_arr[j][i]["cf_arr"].append(wind_data[0,min(min_py,py),min(min_px,px)])
                    #cf_arr[j][i]["area_cell_avg"]
                    #cf_arr[j][i]["area_cell_avg"] += (((111.32 * (math.cos((lat*math.pi)/180)))*0.25)**2)/(N**2) # store est_avg_area_of_cell_ij 
                    

# calculate land proportions per cell
# calculate c_avg per cell
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            tot = len(cf_arr[j][i]["cf_arr"]) # total points in the cell
            cf_arr[j][i]["area_cf_0.1"]= 0
            cf_arr[j][i]["area_cf_0.2"]= 0
            cf_arr[j][i]["area_cf_0.3"]= 0
            cf_arr[j][i]["area_cf_0.4"]= 0
            cf_arr[j][i]["area_cf_0.5"]= 0
            cf_arr[j][i]["avg_cf_0.1"]= 0
            cf_arr[j][i]["avg_cf_0.2"]= 0
            cf_arr[j][i]["avg_cf_0.3"]= 0
            cf_arr[j][i]["avg_cf_0.4"]= 0
            cf_arr[j][i]["avg_cf_0.5"]= 0
            for cf_ij in cf_arr[j][i]["cf_arr"]:
                if  cf_ij <= 0.1:
                    cf_arr[j][i]["area_cf_0.1"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.1"] += cf_ij
                    #continue
                elif cf_ij >0.1 and cf_ij <=0.2:
                    cf_arr[j][i]["area_cf_0.2"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.2"] += cf_ij
                elif cf_ij >0.2 and cf_ij <=0.3:
                    cf_arr[j][i]["area_cf_0.3"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.3"] += cf_ij
                elif cf_ij >0.3 and cf_ij <=0.4:
                    cf_arr[j][i]["area_cf_0.4"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.4"] += cf_ij
                elif cf_ij >0.4:
                    cf_arr[j][i]["area_cf_0.5"] += 1/tot
                    cf_arr[j][i]["avg_cf_0.5"] += cf_ij

#finish cf_avg per cell
eps = 1e-10 #We turn the 0 into a number that is almost zero 
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            cf_arr[j][i]["avg_cf_0.1"] = cf_arr[j][i]["avg_cf_0.1"] / (tot * cf_arr[j][i]["area_cf_0.1"]+eps)
            cf_arr[j][i]["avg_cf_0.2"] = cf_arr[j][i]["avg_cf_0.2"] / (tot * cf_arr[j][i]["area_cf_0.2"]+eps)
            cf_arr[j][i]["avg_cf_0.3"] = cf_arr[j][i]["avg_cf_0.3"] / (tot * cf_arr[j][i]["area_cf_0.3"]+eps)
            cf_arr[j][i]["avg_cf_0.4"] = cf_arr[j][i]["avg_cf_0.4"] / (tot * cf_arr[j][i]["area_cf_0.4"]+eps)
            cf_arr[j][i]["avg_cf_0.5"] = cf_arr[j][i]["avg_cf_0.5"] / (tot * cf_arr[j][i]["area_cf_0.5"]+eps)

# Final average per wind technology type
cf_T_01 = 0
cf_T_02 = 0
cf_T_03 = 0
cf_T_04 = 0
cf_T_05 = 0

tot_01_weights = 0 # total area for cf_0.1 interval
tot_02_weights = 0 # total area for cf_0.2 interval
tot_03_weights = 0 # total area for cf_0.3 interval
tot_04_weights = 0 # total area for cf_0.4 interval
tot_05_weights = 0 # total area for cf_0.5 interval

# modify by land area availabilty per square (from atlite) 
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            tot_01_weights += cf_arr[j][i]["area_cf_0.1"]*srb[j,i]
            tot_02_weights += cf_arr[j][i]["area_cf_0.2"]*srb[j,i]
            tot_03_weights += cf_arr[j][i]["area_cf_0.3"]*srb[j,i]
            tot_04_weights += cf_arr[j][i]["area_cf_0.4"]*srb[j,i]
            tot_05_weights += cf_arr[j][i]["area_cf_0.5"]*srb[j,i]


# Calculate average capacity factor for each tech
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            cf_T_01 += cf_arr[j][i]["area_cf_0.1"] *srb[j,i] * cf_arr[j][i]["avg_cf_0.1"] / tot_01_weights
            cf_T_02 += cf_arr[j][i]["area_cf_0.2"] *srb[j,i] * cf_arr[j][i]["avg_cf_0.2"] / tot_02_weights
            cf_T_03 += cf_arr[j][i]["area_cf_0.3"] *srb[j,i] * cf_arr[j][i]["avg_cf_0.3"] / tot_03_weights
            cf_T_04 += cf_arr[j][i]["area_cf_0.4"] *srb[j,i] * cf_arr[j][i]["avg_cf_0.4"] / tot_04_weights
            cf_T_05 += cf_arr[j][i]["area_cf_0.5"] *srb[j,i] * cf_arr[j][i]["avg_cf_0.5"] / tot_05_weights

area_01_weights = 0 # total area for cf_0.1 interval
area_02_weights = 0 # total area for cf_0.2 interval
area_03_weights = 0 # total area for cf_0.3 interval
area_04_weights = 0 # total area for cf_0.4 interval
area_05_weights = 0 # total area for cf_0.5 interval


#A_cell = 408.5 #0.25*0.25 degrees = 111.1/4 = 27.7775 * 27.7775  
for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            area_01_weights += cf_arr[j][i]["area_cf_0.1"]
            area_02_weights += cf_arr[j][i]["area_cf_0.2"]
            area_03_weights += cf_arr[j][i]["area_cf_0.3"]
            area_04_weights += cf_arr[j][i]["area_cf_0.4"]
            area_05_weights += cf_arr[j][i]["area_cf_0.5"]

# Calculate max capacity per tech (based on available land for that CF interval in each cell)
cap_T_01 = 0
cap_T_02 = 0
cap_T_03 = 0
cap_T_04 = 0
cap_T_05 = 0

for j in range(len(cf_arr)):
    for i in range(len(cf_arr[j])):
        if type(cf_arr[j][i]) == float:
            continue
        else:
            cap_T_01 += cf_arr[j][i]["area_cf_0.1"] * srb[j,i] * cf_arr[j][i]["area_cell_avg"] 
            cap_T_02 += cf_arr[j][i]["area_cf_0.2"] * srb[j,i] * cf_arr[j][i]["area_cell_avg"] 
            cap_T_03 += cf_arr[j][i]["area_cf_0.3"] * srb[j,i] * cf_arr[j][i]["area_cell_avg"] 
            cap_T_04 += cf_arr[j][i]["area_cf_0.4"] * srb[j,i] * cf_arr[j][i]["area_cell_avg"] 
            cap_T_05 += cf_arr[j][i]["area_cf_0.5"] * srb[j,i] * cf_arr[j][i]["area_cell_avg"] 

Modify the path for the df.to_csv to match the name of the input TIFF (i.e. Country code + IEC1, IEC2, IEC3)

In [ ]:
df = pd.DataFrame()
df["TotalAnnualMaxCapacity"] = [cap_T_01.item(0), cap_T_02.item(0), cap_T_03.item(0), cap_T_04.item(0), cap_T_05.item(0)]
df["Capacity Factor"] = [cf_T_01.item(0), cf_T_02.item(0), cf_T_03.item(0), cf_T_04.item(0), cf_T_05.item(0)]
df.to_csv('/Users/Land_eligibility_output_RS_IEC3_AG.csv',index= False)

Normalizing the time series capacity factors from Atlite to match the average CF calculated above. Change the input file

In [ ]:
cf_norm_wind_ba = pd.read_csv('/Users/wind_rs_2.csv') #here it is enough to specify the path and file name to match RS/ME, even though it states cf_norm_wind_ba.
cf_norm_wind_ba['norm'] = cf_norm_wind_ba['wind_cf'].map(lambda x: x/cf_norm_wind_ba['wind_cf'].max())  #map allows us to apply a function to all the values in the series.

target_cf = 0.4245 #Input the target
# For wind in Bosnia you can chose from 0.1532, 0.2506, 0.3434, 0.4411 based on the  IEC1.tif
# For wind in Bosnia you can chose from 0.1553, 0.2502, 0.3461, 0.4452 based on the  IEC2.tif
# For wind in Bosnia you can chose from 0.156, 0.2499, 0.3482, 0.453 based on the  IEC3.tif

# For wind in Bosnia (AG) you can chose from the following (All for the IEAC3.tif), by typing the value in under 'target_cf = '
# 0.1504
# 0.2358
# 0.3446
# 0.4499

# For wind in Montenegro you can chose from 0.1501, 0.2467, 0.341, 0.442 based on the  IEC1.tif
# For wind in Montenegro you can chose from 0.1518, 0.2476, 0.3431, 0.4439 based on the  IEC2.tif
# For wind in Montenegro you can chose from 0.1523, 0.2484, 0.3446, 0.4504 based on the  IEC3.tif

# For wind in Montenegro (AG) you can chose from the following (All for the IEAC3.tif), by typing the value in under 'target_cf = '
# 0.1493
# 0.2462
# 0.3447
# 0.4524

# For wind in Serbia you can chose from 0.1539, 0.2504, 0.3309, 0.4297 based on the  IEC1.tif
# For wind in Serbia you can chose from 0.1577, 0.2476, 0.3368, 0.4362 based on the  IEC2.tif
# For wind in Serbia you can chose from 0.1595, 0.2495, 0.3479, 0.4378 based on the  IEC3.tif

# For wind in Serbia (AG) you can chose from the following (All for the IEAC3.tif), by typing the value in under 'target_cf = '
# 0.1623
# 0.2576
# 0.3462
# 0.4245

guess = target_cf / cf_norm_wind_ba['norm'].mean()
cf_norm_wind_ba['actual_cf'] = cf_norm_wind_ba['norm']
while True: #(target_cf - actual_cf) <= 0.001:
    cf_norm_wind_ba['actual_cf'] = cf_norm_wind_ba['actual_cf'] * guess
    print(actual_cf)
    cf_norm_wind_ba['actual_cf'] = cf_norm_wind_ba['actual_cf'].map(lambda x: np.where(x > 1, 1, x))
    actual_cf = cf_norm_wind_ba['actual_cf'].mean()
    guess = target_cf / actual_cf 
    if abs(target_cf - actual_cf) <= 0.00001:
        break
cf_norm_wind_ba[['date','actual_cf']].to_csv('/Users/wind_rs_2_out_0.5_AG.csv') #here it is enough to specify the path and file name to match RS/ME, even though it states cf_norm_wind_ba.



In [ ]:
 cf_norm_wind_ba['actual_cf'].mean()

In [ ]:
 cf_norm_wind_ba['actual_cf'].max()